In [1]:
from vllm import SamplingParams

## vllm vs. sglang

- vllm: A high-throughput and memory-efficient inference and serving engine for LLMs
- sglang: SGLang is a fast serving framework for large language models and vision language models.
- vllm/sglang is dynamic batch to inference,
    - Validation datasets are sent to inference engines as a whole batch, which will schedule the memory themselves.

### dp vs. tp

- 似乎有一个限制
    - Total number of attention heads (xx) must be divisible by tensor parallel size (4)
    - qwen2.5-7b: 28 attention heads
        - 2卡，4卡，不能 8 卡；

## Parameters

## vllm

- `LLM`
    - `--max-model-len`: Model context length. If unspecified, will be automatically derived from the model config.
        - `max_seq_len`
        - `Qwen/Qwen2.5-7B-Instruct-1M` (config.json, `max_position_embeddings: 1010000`)
    - `max_num_seqs`=256,  # 控制批处理中的最大序列数（batch size）
    - `max_num_batched_tokens`=4096,  # 控制批处理中的最大token数
- `SamplingParams`
    - `max_tokens`: Maximum number of tokens to generate per output sequence.
    - `stop`, ` stop_token_ids`
        - `stop=stop_condition`
     
```python
llm = LLM('Qwen/Qwen2.5-7B-Instruct')
llm.llm_engine.scheduler_config.max_model_len # 32768
llm.llm_engine.scheduler_config.max_num_seqs # 256
llm.llm_engine.scheduler_config.max_num_batched_tokens # 32768
```

### sglang

- https://docs.sglang.ai/backend/server_arguments.html
- https://docs.sglang.ai/backend/offline_engine_api.html